In [3]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from copy import copy

Collaborator: Kaleem

# Problem 1

## A model for the dynamics 
Stock index follows the Black-Scholes dynamics, which assumes GBM
$$\mathrm{d}S_t=R_{grow}S_t\mathrm{d}t+\sigma S_t\mathrm{d}W_t$$
- $\sigma=0.4$, volatility
- $S_0=100$, time-0 index level 
- $r=0$, risk-free interest rate
- $W_t$, Brownian motion

Logarithm of stock index $X=\log S$ follows dynamics of
$$\mathrm dX_t=\nu\mathrm dt+\sigma\mathrm dW_t$$
- $\nu:=R_{grow}-\sigma^{2}/2$

`1a` uses trinomial tree to approximate diffusion dynamics. A trinomial tree is more flexible model than a binomial tree, so we don't run into the situation where we don't have enough free parameters to match both mean and variance. Definitions for up, middle, down probabilities are
- $\mathbb{P}(X_{t+\Delta t}=X_t+\Delta x)=p_u$
- $\mathbb{P}(X_{t+\Delta t}=X_t)=p_m$
- $\mathbb{P}(X_{t+\Delta t}=X_t-\Delta x)=p_d$
- $\Delta t:=T/N$, time step
- $\Delta x=\sigma \sqrt{3\Delta t}$, up/down move size, picked for accuracy reasons

By matching the **diffusion model**
$$X_{t+\Delta t}-X_{t}=\int_{t}^{t+\Delta t}\nu\mathrm{d}s+\int_{t}^{t+\Delta t}\sigma\mathrm{d}W_{s}=\nu\Delta t+\sigma\Delta W$$
to the **tree model** on mean and variance,
- $\nu\Delta t=\mathbb{E}_t(X_{t+\Delta t}-X_t)=p_u\Delta x+p_m0+p_d(-\Delta x)$
- $\sigma^2\Delta t=\mathrm{Var}_t(X_{t+\Delta t}-X_t)=p_u(\Delta x)^2+p_m(0)^2+p_d(-\Delta x)^2-(\nu\Delta t)^2$
- $p_u+p_m+p_d=1$

we get
\begin{aligned}
&p_{u} =\frac{1}{2}\bigg[\frac{\sigma^{2}\Delta t+\nu^{2}(\Delta t)^{2}}{(\Delta x)^{2}}+\frac{\nu\Delta t}{\Delta x}\bigg] \\
&p_{m} =1-\frac{\sigma^{2}\Delta t+\nu^{2}(\Delta t)^{2}}{(\Delta x)^{2}} \\
&p_{d} =\frac{1}{2}\bigg[\frac{\sigma^{2}\Delta t+\nu^{2}(\Delta t)^{2}}{(\Delta x)^{2}}-\frac{\nu\Delta t}{\Delta x}\bigg] 
\end{aligned}

## Contracts to be priced
1. `a`, up-and-out barrier put option
1. `b`, up-and-in barrier put option
1. `c`, continuously-monitored barrier option

In [4]:
class UpAndOutPut:

    def __init__(self, K, T, barrier, observationinterval):
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

In [5]:
# hw1contract = UpAndOutPut(K=95, T=0.25, barrier=107, observationinterval=0.02)
hw1contract = UpAndOutPut(K=95, T=0.25, barrier=114, observationinterval=0.02)

In [6]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [7]:
hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [8]:
class TreeEngine:

    def __init__(self, N):
        self.N = N

    def price_upandout(self, dynamics, contract, discrete_monitoring=True):

        deltat = contract.T / self.N # time step
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5) # space step

        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)
        # Sgrid: an array of stock prices S_T at expiry, ordered from highest to lowest
        # SMALLER indexes in this array correspond to HIGHER S
        # S = exp(X). deltax = X[i+1]-X[i]

        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            raise ValueError("This value of N fails to place the observation dates in the tree.")

        nu = dynamics.rGrow - 0.5 * dynamics.sigma ** 2
        Pu = 0.5 * ((dynamics.sigma ** 2 * deltat + nu ** 2 * deltat ** 2) / (deltax ** 2) 
                    + nu * deltat / deltax)
        Pd = 0.5 * ((dynamics.sigma ** 2 * deltat + nu ** 2 * deltat ** 2) / (deltax ** 2) 
                    - nu * deltat / deltax)
        Pm = 1 - Pu - Pd

        optionprice = np.maximum(contract.K-Sgrid, 0)   #an array of time-T option prices, 2N+1 elements

        #Next, induct backwards to time 0, updating the optionprice array
        #Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]
        if discrete_monitoring:
            for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat:
                Sgrid = Sgrid[1:-1]
                optionprice = np.exp(-dynamics.r * deltat) * (Pu * optionprice[:-2] + Pm * optionprice[1:-1] + Pd * optionprice[2:])
                # check if monitoring date (caveat for floating point)
                if min(abs(t % contract.observationinterval),
                       abs(t % contract.observationinterval 
                         - contract.observationinterval)) < 1e-8:
                    # knock-out/ knock-in check
                    optionprice = np.where(Sgrid >= contract.barrier, 0, optionprice)
                    # print(optionprice)

        return optionprice[0]
        #The [0] is assuming that we are shrinking the optionprice array in each iteration of the loop,
        #until finally there is only 1 element in the array.
        #If instead you are keeping unchanged the size of the optionprice array in each iteration,
        #then you need to change the [0] to a different index.


Barrier option in general:

For accuracy reasons, the procedure is to put $\log H$ halfway between the $j$th and $(j+1)$th log-price levels:
$$\log S_0+(j+0.5)\Delta x=\log H$$
so, with the constraint on $\Delta x \approx \sigma\sqrt{3\Delta t}$, we take
$$j=\left\lceil\frac{\log(H/S_0)}{\sigma\sqrt{3\Delta t}}-0.5\right\rceil $$

### `1a`: up-and-out put

Payout of up-and-out put
$$(K-S_T)^+\mathbf{1}_{\max_{t\in\mathcal{T}}S_t<H}$$
- $S$, stock price 
- $T=0.25$, expiry
- $K=95$, strike
- $H=114$, knock-out barrier
- $\mathcal{T}\subseteq[0,T]$, observation times (`1a` discrete, `1b` continuous)

Pricing of up-and-out European put
$$C_{n}^{j}=e^{-r\Delta t}[p_{u}C_{n+1}^{j+1}+p_{m}C_{n+1}^{j}+p_{d}C_{n+1}^{j-1}]\mathbf{1}_{\mathrm{no~knock-out~at~time~}t_{n}}$$

Algorithmic detail:
- Start from expiry option prices 
- Induct backwards in time, shrinking the number of nodes by 2 each time
- During the induction step, check that no knockout at time $t_n$, i.e. option price is set to zero whenever the barrier level is exceeded

In [9]:
hw1tree=TreeEngine(N=1000) # insufficient
# hw1tree=TreeEngine(N=5000)

up_and_out_put_price = hw1tree.price_upandout(hw1dynamics, hw1contract)
print(f"Price of an Up and Out Put Option: {up_and_out_put_price}")

Price of an Up and Out Put Option: 5.301546106266176


### `1b`: up-and-in put

In [10]:
class UpAndInPut:

    def __init__(self, K, T, barrier, observationinterval):
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

Black-Scholes Vanilla Call pricing:
$$f(S):=(S-K)^{+}$$
$$C(S,t)=C^{BS}(S,t,K,T,r-q,r,\sigma)=S_0N(d_1)-Ke^{-rT}N(d_2)$$
where
$$d_{1,2}:=d_{+,-}:=\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T}\pm\frac{\sigma\sqrt T}2$$
In our case $t=0$, $r=0$.

In [11]:
class VanillaCall:

    def __init__(self, S, K, r, sigma, T):
        self.S = S # S_0 stock price
        self.K = K
        self.r = r
        self.sigma = sigma
        self.T = T
    
    def get_price(self):
        d1 = ((np.log(self.S / self.K) + self.r * self.T) / (self.sigma * np.sqrt(self.T))              
              + 0.5 * self.sigma * np.sqrt(self.T))
        d2 = ((np.log(self.S / self.K) + self.r * self.T) / (self.sigma * np.sqrt(self.T))              
              - 0.5 * self.sigma * np.sqrt(self.T))
        call_price = self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        
        return call_price

Black-Scholes Vanilla Put pricing:
$$f(S):=(K-S)^{+}$$
$$P(S,t)=P^{BS}(S,t,K,T,r-q,r,\sigma)=Ke^{-rT}N(-d_2) - S_0N(-d_1)$$
where
$$d_{1,2}:=d_{+,-}:=\frac{\log(S_0e^{rT}/K)}{\sigma\sqrt T}\pm\frac{\sigma\sqrt T}2$$
In our case $t=0$, $r=0$.

In [12]:
class VanillaPut:

    def __init__(self, S, K, r, sigma, T):
        self.S = S # S_0 stock price
        self.K = K
        self.r = r
        self.sigma = sigma
        self.T = T
    
    def get_price(self):
        d1 = ((np.log(self.S / self.K) + self.r * self.T) / (self.sigma * np.sqrt(self.T))              
              + 0.5 * self.sigma * np.sqrt(self.T))
        d2 = ((np.log(self.S / self.K) + self.r * self.T) / (self.sigma * np.sqrt(self.T))              
              - 0.5 * self.sigma * np.sqrt(self.T))
        put_price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)
        
        return put_price

Up-and-in options are a type of exotic option that is often made available through specialized brokers to high-end clients in the over-the-counter (OTC) markets. The option features both a strike price and a barrier level. As the name suggests, the buyer of the option will benefit once the price of the underlying rises high enough to reach (knock-in) the designated barrier price level. Otherwise, the option will expire worthless.
1. An up-and-in option will give the holder the right to exercise when the barrier price level is reached or exceeded, depending on the structuring.
1. Knock-in options can be either up-and-in or down-and-in. This implies whether the price will rise or fall to meet the barrier price level. The barrier price, when crossed, makes the option available for exercise. 

Payout of up-and-in put
$$C(\text{up-and-out put}) + C(\text{up-and-in put}) = C(\text{vanilla put})$$

Explanation:
- Why we can't naively flip the sign? It might be tempting to simply flip the sign of the option price as compared to the barrier. However, this will result in the entire tree to have too many zeros
- The equation works for the following reason. A vanilla put is always exercisable by definition, which is different from getting a positive payoff. Whether an exotic barrier option is exercisable or not determines whether the holder is eligible to taking the payoff. An up-and-out put is exercisable if the stock price has never exceeded the barrier, but an up-and-in put is exercisable if the stock price has always stayed in. These two cases are disjoint, so them together makes up for the pricing of the vanilla option.

In [13]:
hw1tree=TreeEngine(N=1000) # insufficient
# hw1tree=TreeEngine(N=5000)

hw1vanilla = VanillaPut(S=100, K=95, r=0, sigma=0.4, T=0.25)
vanilla_put_price = hw1vanilla.get_price()
up_and_in_put_price = vanilla_put_price - up_and_out_put_price
print(f"Price of an Up and Out Put Option: {up_and_in_put_price}")

Price of an Up and Out Put Option: 0.21799495741079866


### `1c`: continuous monitoring

`c1`: Smaller than, because options are more likely to get knocked out to zero price, reducing the backtracking average.

`c2`: The continuously monitored barrier option can be replicated by a portfolio of T-expiry options, long 1 plain vanilla put struck at 95, and short α plain vanilla calls struck at 136.8. 
$$(95-S_{0.25})^+\mathbf{1}(\max_{0\leq t\leq0.25}S_t<114)$$
- Long 1 vanilla put payoff $(95-S_{0.25})^+$
- Short $\alpha$ vanilla call payoff $(S_{0.25}- 136.8)^+$

At expiry, the replication automatically works because if the barrier was never exceeded, the call pays nothing and one only gets vanilla put. On the other hand, if the barrier has been crossed, up till that point the option acts like a vanilla put, and after that the holder exit the portfolio, which is the same behavior as the barrier put option. 

At the point in time where the stock price crosses the threshold, to replicate by balancing out the risks 
$$1\cdot (95-S) -\alpha \cdot (S-136.8)=0$$
Plugging in $S=114$, we have $\alpha =0.833$.

In [14]:
alpha = (95 - 114) / (114 - 136.8)
put_price_0 = VanillaPut(S=100, K=95, r=0, sigma=0.4, T=0.25).get_price()
call_price_0 = VanillaCall(S=100, K=136.8, r=0, sigma=0.4, T=0.25).get_price()
print(f"The time-0 value of the continuously-monitored barrier option is: {put_price_0 - alpha * call_price_0}")

The time-0 value of the continuously-monitored barrier option is: 5.031526430574063


# Problem 2

In [15]:
# uses the same GBMdynamics class as in Problem 1

In [16]:
class CallOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

In [17]:
class AnalyticEngine:

    def __init__(self):
        pass

    def BSpriceCall(self, dynamics, contract):
        # ignores contract.price if given, because this function calculates price based on the dynamics

        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        return np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))

    def IV(self, dynamics, contract):
        # ignores dynamics.sigma, because this function solves for sigma.

        if contract.price is None:
            raise ValueError('Contract price must be given')

        df = np.exp(-dynamics.r*contract.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-contract.K)*df])
        C = contract.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan

        dytry = copy(dynamics)
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        sigma_try = 0.2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)>C:
            sigma_try /= 2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)<C:
            sigma_try *= 2
        hi = sigma_try
        lo = hi/2
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above.
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval,
        # for purposes of this homework, you may either (A) write your own bisection algorithm,
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).
        # However, if you are able to bound the solution (of a single-variable problem),
        # then bisection or Brent will be more reliable.

        # Option B: use scipy.optimize.bisect
        # Need to pass the function that takes the implied volatility as a single argument
        # Define a function for the call price as a function of sigma
        def func(sigma):
            return self.BSpriceCall(dytry.update_sigma(sigma), contract) - C

        impliedVolatility = bisect(func, lo, hi)

        return impliedVolatility


### `2a`: time-0 Black-Scholes IV
- for a European call on $S$ at 0.5-year expiry 
- for a European call on $S$ at 1-year expiry 

In [18]:
#Test the BSpriceCall function
hw1analytic = AnalyticEngine()

# new code
dynamics1 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract1 = CallOption(K=100, T=0.5)
callprice1 = hw1analytic.BSpriceCall(dynamics1, contract1)
print(callprice1)

dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract2 = CallOption(K=100, T=1)
callprice2 = hw1analytic.BSpriceCall(dynamics2,contract2)
print(callprice2)

11.246291601828489
15.851941887820608


In [19]:
#Test the IV function
contract1.price = 11.25
contract2.price = 12
hw1IV1 = hw1analytic.IV(dynamics1,contract1)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing
hw1IV2 = hw1analytic.IV(dynamics2,contract2)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing
print(f"IV for a European call on $S$ at 0.5-year expiry: {hw1IV1}")
print(f"IV for a European call on $S$ at 1-year expiry: {hw1IV2}")

IV for a European call on $S$ at 0.5-year expiry: 0.40013278092228577
IV for a European call on $S$ at 1-year expiry: 0.3019384309925955


### `2b`: IV average
- If we assume the pricing for a European call at 0.75-year expiry is the arithmetic average of one at 1-year expiry and one at 0.5-year expiry

In [26]:
hw1IV3 = (hw1IV1 + hw1IV2) / 2
print(f"IV for a European call on $S$ at 0.75-year expiry: {hw1IV3}")

IV for a European call on $S$ at 0.75-year expiry: 0.35103560595744066


In [28]:
dynamics3 = GBMdynamics(sigma=hw1IV3, rGrow=0, S=100, r=0)
contract3 = CallOption(K=100, T=0.75)
pricing3 = hw1analytic.BSpriceCall(dynamics3,contract3)
print(f"Pricing of a European call on $S$ at 0.75-year expiry: {pricing3}")

Pricing of a European call on $S$ at 0.75-year expiry: 12.081533286253702


### `2c`: arbitrage

Assume the $T=0.75$ is overpriced than $T=0.5$ or $T=1$ option. So our portfolio consists of short 2 of $C(S,K,T=0.75)$ and long 1 of $C(S,K,T=0.5)$ and long 1 of $C(S,K,T=1)$. A profit is held at the beginning.

At expiry, since our algorithm assumes a stock dynamics guided by the Black-Scholes model, the expiry payoff is non-negative in all cases. This results in an arbitrage.
